In [2]:
import torch
import torch.nn.functional as F  
torch.manual_seed(1337)
B,T,C = 1,8,2
x = torch.randn(B,T,C)
x.shape

torch.Size([1, 8, 2])

In [3]:
xbow = torch.zeros(B,T,C)
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # icluye el t actual
        xbow[b,t] = torch.mean(xprev,0)


In [4]:
x

tensor([[[ 0.1808, -0.0700],
         [-0.3596, -0.9152],
         [ 0.6258,  0.0255],
         [ 0.9545,  0.0643],
         [ 0.3612,  1.1679],
         [-1.3499, -0.5102],
         [ 0.2360, -0.2398],
         [-0.9211,  1.5433]]])

In [5]:
xbow

tensor([[[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]]])

In [9]:
torch.manual_seed(42)
a = torch.ones(3,3)
#add zeros in the diagonal
a = torch.tril(torch.ones(3,3)) 
a = a / torch.sum(a,1,keepdim=True)

b = torch.randint(0,10,(3,2)).float()
c = a @ b
print("a:= ")
print(a)
print("---")
print("b:= ")
print(b)
print("---")
print("c:= ")
print(c)
print("---")

a:= 
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
---
b:= 
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
---
c:= 
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])
---


In [10]:
wei = torch.tril(torch.ones(T,T))
wei = wei / torch.sum(wei,1,keepdim=True)
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [11]:
xbow2 = wei @ x
xbow2

tensor([[[ 0.1808, -0.0700],
         [-0.0894, -0.4926],
         [ 0.1490, -0.3199],
         [ 0.3504, -0.2238],
         [ 0.3525,  0.0545],
         [ 0.0688, -0.0396],
         [ 0.0927, -0.0682],
         [-0.0341,  0.1332]]])

In [12]:
torch.allclose(xbow,xbow2)

True

In [13]:
tril = torch.tril(torch.ones(T,T))
tril

tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])

In [15]:
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril== 0, float('-inf'))
wei = F.softmax(wei,-1)
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [22]:
torch.manual_seed(1337)
B,T,C = 4,8,2
x = torch.randn(B,T,C)

head_size = 16
key = torch.nn.Linear(C, head_size,bias=False)
query = torch.nn.Linear(C, head_size,bias=False)
value = torch.nn.Linear(C, head_size,bias=False)
k = key(x)
q = query(x)

tril = torch.tril(torch.ones(T,T))
wei = torch.zeros((T,T))
wei = q @ k.transpose(-2,-1) * C ** -0.5 # (B,T,H) @ (B,H,T) = (B,T,T)
wei = wei.masked_fill(tril== 0, float('-inf'))
wei = F.softmax(wei,-1)
v = value(x)
out = wei @ v
wei

tensor([[[1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00],
         [5.5992e-01, 4.4008e-01, 0.0000e+00, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00],
         [3.2197e-01, 2.0162e-01, 4.7641e-01, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00],
         [1.6397e-01, 8.1458e-02, 2.9607e-01, 4.5850e-01, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00],
         [2.0508e-01, 3.0070e-01, 1.8936e-01, 1.8076e-01, 1.2409e-01,
          0.0000e+00, 0.0000e+00, 0.0000e+00],
         [6.0019e-02, 1.2732e-01, 2.9082e-02, 1.6934e-02, 5.5211e-02,
          7.1144e-01, 0.0000e+00, 0.0000e+00],
         [1.4085e-01, 1.0251e-01, 1.7444e-01, 2.0379e-01, 1.6895e-01,
          6.6888e-02, 1.4256e-01, 0.0000e+00],
         [2.2274e-02, 1.0857e-01, 8.2257e-03, 4.0054e-03, 8.0379e-03,
          7.2572e-01, 2.1608e-02, 1.0156e-01]],

        [[1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.00

In [20]:
q.shape, k.shape

(torch.Size([4, 8, 16]), torch.Size([4, 8, 16]))

In [23]:
torch.softmax(torch.tensor([0.1,-0.2,0.3,-0.2,0.5]),dim=-1)

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])

In [24]:
torch.softmax(torch.tensor([0.1,-0.2,0.3,-0.2,0.5])*8,dim=-1)


tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])

In [27]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2,-1) * head_size ** -0.5

In [28]:
k.var(), q.var(), wei.var()

(tensor(0.9213), tensor(0.9248), tensor(0.8903))